In [2]:
import os
__curdir__ = os.getcwd()

import sys
sys.path.append(os.path.join(__curdir__, "../src"))
from utils import get_device

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

device = get_device()

# Using Eagle 7Bn
> https://huggingface.co/RWKV/HF_v5-Eagle-7B

In [ ]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "RWKV/HF_v5-Eagle-7B"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code = True,
    quantization_config = bnb_config,
    device_map = device
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("RWKV/HF_v5-Eagle-7B", trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/RWKV/HF_v5-Eagle-7B:
- tokenization_rwkv_world.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [7]:
def generate_prompt(instruction, input=""):
    instruction = instruction.strip().replace('\r\n','\n').replace('\n\n','\n')
    input = input.strip().replace('\r\n','\n').replace('\n\n','\n')
    if input:
        return f"""Instruction: {instruction}

Input: {input}

Response:"""
    else:
        return f"""User: hi

Assistant: Hi. I am your assistant and I will provide expert full response in full details. Please feel free to ask any question and I will always answer it.

User: {instruction}

Assistant:"""

In [8]:
text = "介绍一下大熊猫"
prompt = generate_prompt(text)

In [9]:
prompt

'User: hi\n\nAssistant: Hi. I am your assistant and I will provide expert full response in full details. Please feel free to ask any question and I will always answer it.\n\nUser: 介绍一下大熊猫\n\nAssistant:'

In [11]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)
output = model.generate(inputs["input_ids"], max_new_tokens=128, do_sample=True, temperature=1.0, top_p=0.3, top_k=0, )
print(tokenizer.decode(output[0].tolist(), skip_special_tokens=True))

: 

: 